<a href="https://colab.research.google.com/github/GurkiratSarna/CE888-Decision-Making-UoE/blob/master/Lab_5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [14]:
#Upload file in sample_data folder
#https://github.com/GurkiratSarna/CE888-Decision-Making-UoE/blob/master/Lab_5/jester-data-1.csv
df = pd.read_csv("jester-data-1.csv", header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [15]:
#Drop first column because it shows the number of jokes that have a score. i.e. not 99 in that row.
df=df.iloc[:,1:]
print(df)

         1      2      3      4     5    ...    96     97     98     99     100
0      -7.82   8.79  -9.66  -8.16 -7.52  ...  99.00  -5.63  99.00  99.00  99.00
1       4.08  -0.29   6.36   4.37 -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
2      99.00  99.00  99.00  99.00  9.03  ...  99.00  99.00  99.00  99.00  99.00
3      99.00   8.35  99.00  99.00  1.80  ...  99.00  99.00  99.00  99.00  99.00
4       8.50   4.61  -4.17  -5.39  1.36  ...   1.55   3.11   6.55   1.80   1.60
...      ...    ...    ...    ...   ...  ...    ...    ...    ...    ...    ...
24978   0.44   7.43   9.08   2.33  3.20  ...   9.03   6.55   8.69   8.79   7.43
24979   9.13  -8.16   8.59   9.08  0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24980  99.00  99.00  99.00  99.00 -7.77  ...  99.00  99.00  99.00  99.00  99.00
24981  99.00  99.00  99.00  99.00 -9.71  ...  99.00  99.00  99.00  99.00  99.00
24982   2.43   2.67  -3.98   4.27 -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 100 columns]


***Helper Function***

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [17]:
arr=df.values
new_arr, idx = replace(arr, 0.1)
print(new_arr)

[[-7.82  8.79 -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


***Latent Features***

In [23]:
n_latent_factors = 2

user_ratings = new_arr
print(user_ratings)
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
print(latent_user_preferences)
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
print(latent_item_features)

[[-7.82  8.79 -9.66 ... 99.   99.   99.  ]
 [ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]
[[0.86225947 0.38040204]
 [0.73988689 0.32758759]
 [0.94348571 0.23665182]
 ...
 [0.51371015 0.00440548]
 [0.88175138 0.33088418]
 [0.59634804 0.11559188]]
[[7.20673867e-01 2.33792298e-02]
 [8.90484567e-01 8.65403393e-02]
 [3.70229243e-02 9.63025729e-01]
 [4.18687377e-01 8.17358539e-01]
 [9.46351182e-03 2.22914427e-01]
 [9.90121116e-01 2.67655700e-01]
 [1.00528323e-01 2.77541894e-01]
 [2.81857589e-02 9.46440407e-01]
 [7.33450557e-02 1.03028508e-01]
 [7.87275059e-01 1.34906605e-01]
 [1.05378558e-01 3.96972047e-01]
 [2.18537991e-01 1.89710516e-01]
 [9.58227999e-01 7.24382676e-01]
 [7.85072660e-01 9.60587065e-01]
 [5.28618582e-01 6.64534026e-01]
 [4.97421300e-01 4.60130690e-01]
 [7.92582928e-03 5.61403138e-01]
 [7.65893318e-01 5.

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    
#change nan condition to not equal to 99
#change 10000 modulo operation to 10, for 100 sgd() iterations
def sgd(iterations=300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [0]:
#sgd()
#sgd(100)
#sgd(20) 
'''814.0265993739114
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in multiply
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in subtract
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in square'''
#sgd(15)
'''nan'''
sgd(100)

26.23354558164789


In [0]:
pred = latent_user_preferences.dot(latent_item_features.T)
print(pred)

In [0]:
values = [zip(user_ratings[i], pred[i]) for i in range(pred.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
# the first number is the orginal value from the jester-ata-1.csv dataset, 
# the second number is the predicted value
print(comparison_data)

In [0]:
#Calculate the performance (e.g., MSE) of the algorithm on the validation dataset
error=(pred[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mse = sum(error)/len(error)
mse

***Changing hyper-parameters (learning rate, number of iterations of SVD, number of latent factors)***

In [0]:
# changed factors to 3
n_latent_factors2 = 3

user_ratings2 = new_arr
user_ratings2
# Initialise as random values
latent_user_preferences2 = np.random.random((user_ratings2.shape[0], n_latent_factors2))
latent_item_features2 = np.random.random((user_ratings2.shape[1], n_latent_factors2))

In [0]:
def predict_rating2(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference2 = latent_user_preferences2[user_id]
    item_preference2 = latent_item_features2[item_id]
    return user_preference2.dot(item_preference2)

#changed training to 0.001
def train2(user_id, item_id, rating, alpha=0.001):
    
    #print item_id
    prediction_rating2 = predict_rating2(user_id, item_id)
    err =  prediction_rating2 - rating
    #print err
    user_pref_values2 = latent_user_preferences2[user_id][:]
    latent_user_preferences2[user_id] -= alpha * err * latent_item_features2[item_id]
    latent_item_features2[item_id] -= alpha * err * user_pref_values2
    return err
    

def sgd2(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences2.shape[0]):
            for item_id in range(latent_item_features2.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train2(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
sgd2(30)

In [0]:
pred2 = latent_user_preferences2.dot(latent_item_features2.T)
pred2

In [0]:
values2 = [zip(user_ratings[i], pred2[i]) for i in range(pred2.shape[0])]
comparison_data2 = pd.DataFrame(values2)
comparison_data2.columns = df.columns

In [0]:
comparison_data2

In [0]:
error2=(pred2[idx[0][:],idx[1][:]]-arr[idx[0][:],idx[1][:]])**2
mse2 = sum(error2)/len(error2)
mse2